# Question-answering chatbot
In this example we will be going over the code required to perform Question-answering chatbot. This example uses a BERT to extract features of questions that are then used with Milvus to build a system that can perform the searches.

## Data
This example uses [InsuranceQA Corpus](https://github.com/shuzi/insuranceQA) dataset, which contains 27,413 answers (answer set size is 27,413) with the 3,065,492 running words of answers.

Download location: https://github.com/chatopera/insuranceqa-corpus-zh/tree/release/corpus/pairs

In this example, we use a small dataset that containing 100 pairs of quesiton-answer and you can find it under **data** directory.

## Requirements
- [Milvus 1.0](https://milvus.io/docs/install_milvus.md)
- [Postgresql](https://www.postgresql.org/)
- [Bert](https://github.com/hanxiao/bert-as-service/tree/master)
- requirements.txt

### Install required python packages

In [ ]:
pip install -r requirements.txt

### Start Milvus Server

This demo uses Milvus 1.1.0. Refer to the [Install Milvus](https://milvus.io/docs/milvus_docker-cpu.md) for how to install Milvus docker. 

In [ ]:
! docker run -d --name milvus_cpu_1.1.0 \
-p 19530:19530 \
-p 19121:19121 \
-v /home/$USER/milvus/db:/var/lib/milvus/db \
-v /home/$USER/milvus/conf:/var/lib/milvus/conf \
-v /home/$USER/milvus/logs:/var/lib/milvus/logs \
-v /home/$USER/milvus/wal:/var/lib/milvus/wal \
milvusdb/milvus:1.1.0-cpu-d050721-5e559c

### Install PostgreSQL
For now, Milvus doesn't support to store string type data. Thus, we need a relational database to store questions and answers. In this example, we use [PostgreSQL](https://www.postgresql.org/).

Install by launching Postgres.app: https://www.postgresql.org/download/

### Install and start BERT
#### 1. Download model

In [ ]:
! wget -P model https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
! unzip model/cased_L-12_H-768_A-12.zip -d model/

#### 2. Start BERT service  
BERT service keeps running which causes that we cannot run the next cell. So here we need go to the `model` path and then run the following command.
```shell
cd your_path/model
bert-serving-start -model_dir cased_L-12_H-768_A-12/ -num_worker=2 -max_seq_len=40
```

## Code Overview
### Connecting to Servers

We first start off by connecting to the servers. In this case the Milvus docker container is running on localhost and the port is the default port. And after start starting BERT and Postgresql you can get the ports of them.  
<img style="float: left;" src="./pic/BERT_info.png" width="40%">
<img style="float: right;" src="./pic/Postgresql_info.png" width="60%">

In [ ]:
#Connectings to Milvus, BERT and Postgresql

from milvus import Milvus, IndexType, MetricType, Status
from bert_serving.client import BertClient
import psycopg2

# TODO: Change USER_NAME to your computer user name.
USER_NAME = 'mialee'

milvus = Milvus(host = '127.0.0.1', port = 19530)
bc = BertClient(ip='127.0.0.1', port=5555, check_length=False)
conn = psycopg2.connect(host='localhost', port='5432', user=USER_NAME, password='', database=USER_NAME)
cursor = conn.cursor()

## Create collection and set index
#### 1. Create a collection  
A collection in Milvus is similar to a table in a relational database, and is used for storing all the vectors.  
Required parameters for creating a collection:  
    - `collection_name`: Name of a collection.  
    - `dimension`: BERT generates 728-dimensional vectors.  
    - `index_file_size`: how large each data segmet will be within the collection.      
    - `metric_type`: the distance formula being used to calculate similarity. In this example we are using Inner product (IP).

In [ ]:
TABLE_NAME = 'chatbot'

collection_param = {
            'collection_name': TABLE_NAME,
            'dimension': 768,
            'index_file_size': 1024,  
            'metric_type': MetricType.IP 
            }

status = milvus.create_collection(collection_param)
print(status)

#### 2. Set an index
After creating the collection we want to assign it an index type. This can be done before or after inserting the data. When done before, indexes will be made as data comes in and fills the data segments. In this example we are using IVF_FLAT which requires the 'nlist' parameter. Each index types carries its own parameters. More info about this param can be found [here](https://milvus.io/docs/v1.1.0/index.md#CPU).

In [ ]:
param = {'nlist': 16384}
status = milvus.create_index(TABLE_NAME, IndexType.IVF_FLAT, param)
print(status)

### Create table in Postgre  
PostgresSQL will be used to store the Milvus ID and its corresponding answer and question.

In [ ]:
try:
    sql = "CREATE TABLE if not exists " + TABLE_NAME + " (ids bigint, question text, answer text);"
    cursor.execute(sql)
    conn.commit()
    print("create postgres table successfully!")
except Exception as e:
    print("can't create a postgres table: ", e)

### Processing and Storing QA dataset
#### 1.Generate embeddings
Use BERT to convert questions into vectors. Then normalize and import vectors into Milvus. 

In [ ]:
import pandas as pd
from functools import reduce
import numpy as np

# Get questions and answers.
data = pd.read_csv('data/example.csv')
question_data = data['question'].tolist()
answer_data = data['answer'].tolist()

# Convert quesitons to embeddings and normalize them.
def normaliz_vec(vec_list):
    question_vec = []
    for vec in vec_list:
        square_sum = reduce(lambda x,y:x+y, map(lambda x:x*x ,vec))
        sqrt_square_sum = np.sqrt(square_sum)
        coef = 1/sqrt_square_sum
        vec = list(map(lambda x:x*coef, vec))
        question_vec.append(vec)
    return question_vec

question_vec = bc.encode(question_data)
question_norm_vec = normaliz_vec(question_vec)

#### 2. Insert vectors in Milvus

In [ ]:
status, ids = milvus.insert(collection_name=TABLE_NAME, records=question_norm_vec)

status

#### 3. Import IDs, questions and answers in PostgreSQL
And then import the generated (or specific) IDs and their corresponding questions and answers in PostgreSQL.

In [ ]:
import os 

def record_temp_csv(fname, ids, answer, question):
    with open(fname,'w') as f:
        for i in range(len(ids)):
            line = str(ids[i]) + "|" + question[i] + "|" + answer[i] + "\n"
            f.write(line)

def copy_data_to_pg(table_name, fname, conn, cur):
    fname = os.path.join(os.getcwd(),fname)
    sql = "copy " + table_name + " from '" + fname + "' with CSV delimiter '|';"
    try:
        cursor.execute(sql)
        conn.commit()
        print("insert to pg sucessfully!")
    except Exception as e:
        print("copy data to postgre failed: ", e)
        
DATA_WITH_IDS = 'data/example_with_ids.csv'   

record_temp_csv(DATA_WITH_IDS, ids, answer_data, question_data)
copy_data_to_pg(TABLE_NAME, DATA_WITH_IDS, conn, cursor)

### Searching
#### 1. Process query question and search in Milvus
When searching for a question, we first put the question through the same BERT model to generate embedding. Then we can get a vector and use it to search similar embeddings in Milvus.  
There may not have questions that are similar to the given one. So we can set a threshold value, here we use 0.8, and when the most similar distance retrieved is less than this value, a hint that the system doesn't include the relevant question is returned.

In [ ]:
SEARCH_PARAM = {'nprobe': 32}

question = "Which life insurance is more recommended?"
vector = bc.encode([question])
vector_list = normaliz_vec(vector.tolist())
status, results = milvus.search(collection_name=TABLE_NAME, query_records=vector_list, top_k=5, params=SEARCH_PARAM)

if results[0][0].distance < 0.8:
    print("No similar questions in the database!")
else:
    print(results)

#### 2. Get similar questions
Quering the corresponding questions in PostgreSQL with the returned IDs.

In [ ]:
similar_questions = []
for result in results[0]:
    sql = "select question from " + TABLE_NAME + " where ids = " + str(result.id) + ";"
    cursor.execute(sql)
    rows=cursor.fetchall()
    if len(rows):
        similar_questions.append(rows[0][0])

print(similar_questions)

#### 3. Get the answer
After getting a list of similar questions, choose the one that you feel is closest to yours. Then you can use the question to search for the corresponding answer in PostgreSQL.

In [ ]:
sql = "select answer from " + TABLE_NAME + " where question = '" + similar_questions[0] + "';"
cursor.execute(sql)
rows=cursor.fetchall()

print(rows[0][0])